모듈 ∙ 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping

데이터 불러오기 ∙ 전처리

In [2]:
def filter_df(df): #데이터프레임 필터링
    # 첫 번째 열에서 같은 값을 가진 행의 수를 계산합니다.
    row_counts = df['0'].value_counts()

    # 가장 많은 행의 수를 찾습니다.
    max_row_count = row_counts.max()

    # 가장 많은 행의 수에 해당하는 행만 분류합니다.
    filtered = pd.DataFrame(df[df['0'].isin(row_counts[row_counts == max_row_count].index)])

    return filtered

# 데이터 로드   
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData.csv',low_memory=False)

#데이터 필터링
filter_stock = filter_df(stock_df)
filter_label = filter_stock['18']

# 불필요한 데이터 삭제
filter_stock = filter_stock.drop({'0','1','7','18'},axis=1) #날자, 상승율, 5%이상 상승여부 삭제 

데이터 전처리

In [3]:
scaler = MinMaxScaler()
stock_label = scaler.fit_transform(filter_label.values.reshape(-1, 1))

데이터 분할

In [4]:
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(filter_stock, stock_label, test_size=0.2, random_state=42)

# smote = SMOTE(sampling_strategy='auto',random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)
# X_test_resampled, y_test_resampled = smote.fit_resample(X_test,y_test)

GRU 모델 정의

In [5]:
# GRU 모델 정의
model = tf.keras.Sequential()
model.add(tf.keras.layers.GRU(64, input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=False))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
model.summary()

모델 훈련

In [6]:
# 모델 훈련
model.fit(X_train,y_train,epochs=100,
validation_data=(X_test,y_test),callbacks=[early_stopping])

# 모델 저장
# model.save("GRU_Model.h5")
model.save("GRU_Model_7L_64.h5")

모델 평가

In [7]:
# 모델 로드
loaded_model = tf.keras.models.load_model("GRU_Model_7L_64.h5")

# 모델을 사용하여 주가 상승 여부 예측
test = loaded_model.predict(X_test)

3940/3940 [==============================] - 41s 10ms/step


In [8]:
# 예측 결과를 이진값(0 또는 1)으로 변환
binary_predictions = (test > 0.5).astype(int)

# 예측 결과 출력
correct_predictions = np.equal(binary_predictions, y_test)  # 정확하게 예측한 경우 True, 그렇지 않으면 False
accuracy = np.mean(correct_predictions)  # 정확도 계산

print(f"정확도: {accuracy * 100:.2f}%")

정확도: 96.49%
